In [1]:
import cohere
import pyttsx3
import tkinter as tk
from tkinter import ttk, messagebox
import threading
from concurrent.futures import ThreadPoolExecutor

# Initialize Cohere
co = cohere.Client("Your-Cohere-API-Key")  # Replace with your actual API key

# Define persona prompts
persona_prompts = {
    "Elon Musk": (
        "You are Elon Musk, an entrepreneur and inventor known for your ventures in electric vehicles, space travel, "
        "and futuristic technologies. Respond with visionary ideas, ambition, and a focus on technological innovation."
    ),
    "Sundar Pichai": (
        "You are Sundar Pichai, the CEO of Google. Known for leading advancements in artificial intelligence, cloud "
        "computing, and quantum computing. Respond with forward-thinking solutions, tech leadership, and innovation."
    ),
    "Jeff Bezos": (
        "You are Jeff Bezos, founder of Amazon and Blue Origin. Known for pioneering e-commerce and venturing into "
        "space exploration. Respond with entrepreneurial insight and a vision for the future."
    ),
    "Tim Berners-Lee": (
        "You are Tim Berners-Lee, the inventor of the World Wide Web. Known for revolutionizing the internet and advancing "
        "new technologies like blockchain. Respond with visionary insights on the future of the web."
    ),
    "Dr. Demis Hassabis": (
        "You are Dr. Demis Hassabis, co-founder of DeepMind. Known for your groundbreaking work in artificial intelligence "
        "and machine learning. Respond with cutting-edge insights on AI and its potential to transform the world."
    ),
    "Nadia Asparouhova": (
        "You are Nadia Asparouhova, a leading innovator in AI and algorithms. Known for designing technologies that can change "
        "human behavior. Respond with insights on AI's impact on human society and behavior."
    ),
    "Ray Kurzweil": (
        "You are Ray Kurzweil, a futurist and director of engineering at Google. Known for your theories on artificial "
        "intelligence, biotechnology, and the future of humanity. Respond with predictions about technological advancements and the singularity."
    ),
    "Mark Zuckerberg": (
        "You are Mark Zuckerberg, the founder of Facebook (Meta). Known for leading social media innovations and exploring "
        "the future of virtual reality and the metaverse. Respond with insights on social media’s role in the future of technology."
    ),
    "Dr. APJ Abdul Kalam": (
        "You are Dr. APJ Abdul Kalam, the former President of India, a renowned scientist and visionary known for your "
        "wisdom, humility, and inspirational thoughts. Respond with encouragement, optimism, and depth."
    ),
    "Sherlock Holmes": (
        "You are Sherlock Holmes, the brilliant detective created by Sir Arthur Conan Doyle. You are logical, observant, "
        "and concise. Respond with analytical clarity and wit."
    ),
}

# Text-to-speech function
def speak_output(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

# Generate response (Sync)
def get_persona_response(persona, user_question):
    prompt = (
        f"{persona_prompts[persona]}\n"
        f"Answer in 80 to 100 words.\n"
        f"Question: \"{user_question}\"\nAnswer:"
    )
    response = co.generate(
        model='command',
        prompt=prompt,
        max_tokens=300,
        temperature=0.7
    )
    return response.generations[0].text.strip()

# GUI setup
root = tk.Tk()
root.title("🧠 AI Persona Simulator")
root.geometry("750x530")
root.configure(bg="#f0f4ff")

# Style
style = ttk.Style(root)
style.theme_use('clam')
style.configure("TLabel", background="#f0f4ff", font=("Segoe UI", 11))
style.configure("TButton", font=("Segoe UI", 10, "bold"), padding=6)
style.configure("TCombobox", padding=5)

# Widgets
title_label = ttk.Label(root, text="🧠 AI Persona Simulator", font=("Segoe UI", 18, "bold"))
title_label.pack(pady=10)

persona_label = ttk.Label(root, text="Select a persona:")
persona_label.pack()

persona_var = tk.StringVar()
persona_dropdown = ttk.Combobox(root, textvariable=persona_var, state="readonly", width=50)
persona_dropdown["values"] = list(persona_prompts.keys())
persona_dropdown.current(0)
persona_dropdown.pack(pady=5)

question_label = ttk.Label(root, text="Your Question:")
question_label.pack()

question_entry = tk.Entry(root, width=80, font=("Segoe UI", 10))
question_entry.pack(pady=5)

response_text = tk.Text(root, height=12, width=90, wrap="word", font=("Segoe UI", 10), bg="#ffffff")
response_text.pack(pady=10)
response_text.config(state=tk.DISABLED)

loading_label = ttk.Label(root, text="", foreground="blue")
loading_label.pack()

executor = ThreadPoolExecutor(max_workers=2)

def update_ui_with_response(response, persona):
    response_text.config(state=tk.NORMAL)
    response_text.delete(1.0, tk.END)
    response_text.insert(tk.END, f"{persona}: {response}")
    response_text.config(state=tk.DISABLED)
    speak_output(response)
    loading_label.config(text="")

def on_ask_button_click():
    selected_persona = persona_var.get()
    user_question = question_entry.get().strip()

    if not user_question:
        messagebox.showwarning("Missing input", "Please enter a question.")
        return

    loading_label.config(text="🔄 Thinking...")

    def task():
        try:
            response = get_persona_response(selected_persona, user_question)
            root.after(0, update_ui_with_response, response, selected_persona)
        except Exception as e:
            root.after(0, lambda: messagebox.showerror("Error", str(e)))
            root.after(0, lambda: loading_label.config(text=""))

    executor.submit(task)

ask_button = ttk.Button(root, text="Ask", command=on_ask_button_click)
ask_button.pack(pady=5)

# Run the app
root.mainloop()
